Documentación: https://learn.qiskit.org/course/ch-applications/simulating-molecules-using-vqe

Cálculo de energía del estado base de la molécula LiH en función de la distancia interatómica.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver, NumPyMinimumEigensolverFactory)
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (ElectronicStructureMoleculeDriver, ElectronicStructureDriverType)
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit.opflow import TwoQubitReduction
from qiskit.providers.aer.noise import NoiseModel

Trabajaremos con hidruro de litio (LiH). Se crea un driver para la mólecula en cada cambio de distancia interatómica. Para reducir el número de qubits utilizados, se congela el núcleo y se remueven dos orbitales NO ocupados.

Inicialmente, definimos una función que tome una distancia interatómica y retorne un operador qubit H apropiado.

In [3]:
def get_qubit_op(dist):
    # Definicion de la molecula
    molecule = Molecule(geometry = [["Li", [0.0, 0.0, 0.0]],  # Coordenadas en Angstrom
                                   ["H", [dist, 0.0, 0.0]]],  # dist es el valor a variar
               multiplicity = 1,                              # = 2*spin + 1
               charge = 0,
    )
    
    driver = ElectronicStructureMoleculeDriver(molecule = molecule,
                                               basis = "sto3g",     #STO-3G describe los orbitales electrónicos de la molécula                                  
                                               driver_type = ElectronicStructureDriverType.PYSCF)

    #Propiedades
    properties = driver.run()
    num_particles = (properties
                     .get_property("ParticleNumber")
                     .num_particles)
    num_spin_orbitals = int(properties
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)

    #Se congela el nucleo y se remuevn los orbitales NO ocupados.
    problem = ElectronicStructureProblem(driver,
                                         [FreezeCoreTransformer(freeze_core = True, remove_orbitals = [-3,-2])])

    second_q_ops = problem.second_q_ops()          #Se obtiene la 2da cuantizacion del operador
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles

    #Hamiltoniano del sistema, mapeado con funciones de qiskit
    mapper = ParityMapper()        
    hamiltonian = second_q_ops[0]
    
    #Reduccion a dos qubits
    converter = QubitConverter(mapper, two_qubit_reduction = True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

In [ ]:
#Cálculo con algoritmos clásicos exactos
def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 4.25, 0.05)                   #Se variará la distancia desde 0.5A->4.25A en pasos de 0.25A

#Arreglos para guardar las energías calculadas
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter = 1000)                        #Optimizador estableciendo un número máximo de 1000 iteraciones

#Loop que correr sobre las diferentes distancias
for dist in distances:
    (qubit_op, num_particles, num_spin_orbitals, problem, converter) = get_qubit_op(dist)
    
    result = exact_solver(problem,converter)
    exact_energies.append(result.total_energies[0].real)
    
    init_state = HartreeFock(num_spin_orbitals, num_particles, converter)  #Ansatz del VQE
    var_form = UCCSD(converter,                                            #Aplicación de la variación utilizando UCCSD
                     num_particles,
                     num_spin_orbitals,
                     initial_state = init_state)
    vqe = VQE(var_form, optimizer, quantum_instance=backend)                #Algoritmo VQE
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(f"Interatomic Distance: {np.round(dist, 3)}",
          f"VQE Result: {vqe_result:.5f}",
          f"Exact Energy: {exact_energies[-1]:.5f}")

print("All energies have been calculated")

/home/ccortesp/.local/lib/python3.8/site-packages/qiskit_nature/problems/second_quantization/electronic/electronic_structure_problem.py:93: ListAuxOpsDeprecationWarning: List-based `aux_operators` are deprecated as of version 0.3.0 and support for them will be removed no sooner than 3 months after the release. Instead, use dict-based `aux_operators`. You can switch to the dict-based interface immediately, by setting `qiskit_nature.settings.dict_aux_operators` to `True`.
  second_quantized_ops = self._grouped_property_transformed.second_q_ops()


Interatomic Distance: 0.5 VQE Result: -7.04791 Exact Energy: -7.04791
Interatomic Distance: 0.55 VQE Result: -7.19571 Exact Energy: -7.19572
Interatomic Distance: 0.6 VQE Result: -7.31784 Exact Energy: -7.31784
Interatomic Distance: 0.65 VQE Result: -7.41937 Exact Energy: -7.41937
Interatomic Distance: 0.7 VQE Result: -7.50400 Exact Energy: -7.50400
Interatomic Distance: 0.75 VQE Result: -7.57458 Exact Energy: -7.57458
Interatomic Distance: 0.8 VQE Result: -7.63338 Exact Energy: -7.63338
Interatomic Distance: 0.85 VQE Result: -7.68228 Exact Energy: -7.68228
Interatomic Distance: 0.9 VQE Result: -7.72283 Exact Energy: -7.72283
Interatomic Distance: 0.95 VQE Result: -7.75638 Exact Energy: -7.75638
Interatomic Distance: 1.0 VQE Result: -7.78402 Exact Energy: -7.78402
Interatomic Distance: 1.05 VQE Result: -7.80670 Exact Energy: -7.80670
Interatomic Distance: 1.1 VQE Result: -7.82520 Exact Energy: -7.82520
Interatomic Distance: 1.15 VQE Result: -7.84018 Exact Energy: -7.84018
Interatomic D

In [3]:
#Plots
plt.plot(distances, exact_energies, 'x', label = "Exact Energy")
plt.plot(distances, vqe_energies, 'o', markersize = 3, label = "VQE Energy")
plt.xlabel('Interatomic distance (Angstrom)')
plt.ylabel('Energy')
plt.title('LiH Ground State Energy')
plt.legend(loc = 'upper right')
plt.savefig('lih.png')
plt.show()

NameError: name 'distances' is not defined